<a href="https://colab.research.google.com/github/Harsh-eng568/API_IPL/blob/main/ipl_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import ast

matches = pd.read_csv(r'C:\Users\hband\Downloads\session-datasets-20250222T104901Z-001\session-datasets\datasets-session-17\ipl-matches.csv')
# print(matches.head(2))

def teamsAPI():
    teams =   list(set(list(matches['Team1']) + list(matches['Team2'])))
    team_dict = {
        'teams':teams
    }
    return team_dict

def team_vs_team(team1, team2):
    valid_teams = list(set(list(matches['Team1']) + list(matches['Team2'])))
    if team1 in valid_teams and team2 in valid_teams:
        temp_df = matches[(matches['Team1'] == team1) & (matches['Team2'] == team2) | (matches['Team1'] == team2) & (matches['Team2'] == team1)]
        total_matches = temp_df.shape[0]

        matches_won_team1 = temp_df['WinningTeam'].value_counts()[team1]
        matches_won_team2 = temp_df['WinningTeam'].value_counts()[team2]

        draws = total_matches - (matches_won_team1 + matches_won_team2)
        response = {
            'total_matches': str(total_matches),
             team1 : str(matches_won_team1),
             team2:  str(matches_won_team2),
            'draws': str(draws)
        }
        return response
    else:
        return {'message':'invalid team name'}

def top_player_for_season(year):
    year = str(year)  # Convert to string to match your DataFrame's 'Season' type
    season_data = matches[matches['Season'] == year]

    if season_data.empty:
        return {"error": f"No data found for season {year}"}

    top = season_data['Player_of_Match'].value_counts().idxmax()
    count = season_data['Player_of_Match'].value_counts().max()

    return {
        "Season": year,
        "Player": top,
        "Awards": int(count)
    }

def count_player_of_match(team_name, team_matches):
    count = 0
    for _, row in team_matches.iterrows():
        try:
            if row['Team1'] == team_name:
                players = ast.literal_eval(row['Team1Players'])
            elif row['Team2'] == team_name:
                players = ast.literal_eval(row['Team2Players'])
            else:
                continue  # skip if team is not involved, though unlikely here

            if row['Player_of_Match'] in players:
                count += 1
        except:
            continue  # skip malformed data
    return count


def get_team_record(team_name):
    team_matches = matches[(matches['Team1'] == team_name) | (matches['Team2'] == team_name)]

    total_matches = len(team_matches)
    wins = team_matches[team_matches['WinningTeam'] == team_name].shape[0]
    losses = total_matches - wins
    winning_percentage = (wins / total_matches) * 100 if total_matches > 0 else 0
    largest_margin = team_matches['Margin'].max()

    player_of_match_count = count_player_of_match(team_name, team_matches)

    record = {
        'team': team_name,
        'total_matches': total_matches,
        'wins': wins,
        'losses': losses,
        'winning_percentage': winning_percentage,
        'largest_margin': largest_margin,
        'player_of_match_count': player_of_match_count
    }

    return record

def get_batsman_awards_record(batsman_name):
    count = matches[matches['Player_of_Match'] == batsman_name].shape[0]
    return {
        'batsman': batsman_name,
        'player_of_match_awards': count
    }


def get_bowler_match_count(bowler_name):
    played = matches[
        matches['Team1Players'].apply(lambda players: bowler_name in players) |
        matches['Team2Players'].apply(lambda players: bowler_name in players)
        ]
    total_matches = played.shape[0]
    awards = played[played['Player_of_Match'] == bowler_name].shape[0]

    return {
        'bowler': bowler_name,
        'matches_played': total_matches,
        'player_of_match_awards': awards
    }

def matches_at_venue(venue_name):
    venue_matches = matches[matches['Venue'] == venue_name]
    return {
        'venue': venue_name,
        'total_matches': venue_matches.shape[0],
        'matches': venue_matches[['Date', 'Team1', 'Team2', 'WinningTeam', 'Player_of_Match']].to_dict(orient='records')
    }


def matches_on_date(date_str):
    date_matches = matches[matches['Date'] == date_str]
    if date_matches.empty:
        return {'message': 'No matches found on this date'}

    return {
        'date': date_str,
        'matches': date_matches[['Team1', 'Team2', 'Venue', 'WinningTeam', 'Player_of_Match']].to_dict(orient='records')
    }

def toss_stats(team):
    tosses = matches[matches['TossWinner'] == team]
    total_tosses = tosses.shape[0]
    decisions = tosses['TossDecision'].value_counts().to_dict()

    return {
        'team': team,
        'tosses_won': total_tosses,
        'decisions': decisions
    }

def matches_by_margin(win_by='Wickets', margin=10):
    margin_matches = matches[(matches['WonBy'] == win_by) & (matches['Margin'] == margin)]
    return {
        'total_matches': margin_matches.shape[0],
        'matches': margin_matches[['Date', 'Team1', 'Team2', 'WinningTeam', 'Venue']].to_dict(orient='records')
    }
